In [1]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt

### Load Data

In [47]:
data = pd.read_excel('resources/fall2017data_cleaned_v2.xlsx', delimiter=',')
# data = data.fillna(0)
data
clean_data = data

In [3]:
data.shape

(284, 24)

In [4]:
data.Grade.value_counts()

7.0     44
8.0     41
10.0    41
12.0    39
9.0     39
5.0     37
11.0    37
6.0      5
Name: Grade, dtype: int64

In [90]:
"Other" in list(data.iloc[1])

True

In [109]:
count = 0
for i in range(data.shape[0]):
    if "Other" in list(data.iloc[i]):
        count += 1
print("'Other' is in", str(count), "entries")        

'Other' is in 112 entries


### Data Processing
#### Cast 6th grade morning choices

In [48]:
sixth_graders = clean_data['Grade']  == 5

for choice in ['First', 'Second', 'Third']:
    for i in [1, 2, 3, 4]:
        column_name = 'Period ' + str(i) + ' (' + choice + ' Choice)'
        clean_data.loc[sixth_graders, column_name] = '6thGradeOther'
# clean_data

In [49]:
for choice in ['First', 'Second', 'Third']:
    for i in [1, 2, 3, 4, 7, 8]:
        clean_data = clean_data.iloc[np.where(clean_data['Period ' + str(i) + ' (' + choice + ' Choice)'] != 0)]
print(clean_data.shape)
# clean_data

(284, 24)


In [94]:
clean_data.Grade.value_counts()

7.0     44
8.0     41
10.0    41
12.0    39
9.0     39
5.0     37
11.0    37
6.0      5
Name: Grade, dtype: int64

In [97]:
# count = 0
# for i in range(clean_data.shape[0]):
#     if "Other" in list(clean_data.iloc[i]):
#         count += 1
# print("'Other' is in", str(count), "entries")   

In [98]:
# clean_data.columns[5:]

In [8]:
num_other = {}
for col in clean_data.columns[5:]:
    num_other[col] = np.sum(clean_data[col]=="Other")
num_other

{'Notes': 0,
 'Period 1 (First Choice)': 31,
 'Period 1 (Second Choice)': 38,
 'Period 1 (Third Choice)': 40,
 'Period 2 (First Choice)': 27,
 'Period 2 (Second Choice)': 35,
 'Period 2 (Third Choice)': 41,
 'Period 3 (First Choice)': 24,
 'Period 3 (Second Choice)': 24,
 'Period 3 (Third Choice)': 35,
 'Period 4 (First Choice)': 23,
 'Period 4 (Second Choice)': 33,
 'Period 4 (Third Choice)': 36,
 'Period 7 (First Choice)': 26,
 'Period 7 (Second Choice)': 27,
 'Period 7 (Third Choice)': 36,
 'Period 8 (First Choice)': 25,
 'Period 8 (Second Choice)': 25,
 'Period 8 (Third Choice)': 30}

In [50]:
# clean_data['Period 1 (Third Choice)'].isnull().values.any()

clean_data = clean_data.fillna('Empty')
clean_data

,Timestamp,Email Address,Student Name,Grade,Family Group Leader,Period 1 (First Choice),Period 1 (Second Choice),Period 1 (Third Choice),Period 2 (First Choice),Period 2 (Second Choice),...,Period 4 (First Choice),Period 4 (Second Choice),Period 4 (Third Choice),Period 7 (First Choice),Period 7 (Second Choice),Period 7 (Third Choice),Period 8 (First Choice),Period 8 (Second Choice),Period 8 (Third Choice),Notes
0,2017-06-14 14:26:24.431000,rkolb@icsd.k12.ny.us,Aaron jorgensen,11,Gina kolb,Evolutionary Biology,Evolutionary Biology,Other,HS English (TBA),Geology,...,Economics,Other,Economics,Other,US History and Lit: The 20th Century,US History and Lit: The 20th Century,US History and Lit: The 20th Century,US History and Lit: The 20th Century,US History and Lit: The 20th Century,Other = resource room
1,2017-06-15 08:21:16.889000,simon.warhaft@icsd.k12.ny.us,Abbi Fuller,9,Z,HS PE,Portfolio Photo,Portfolio Photo,Geology,Other,...,Beginning Algebra and Trigonometry,MS/HS PE,Intro to Physics and Chemistry,Latin American Studies,African Studies,Latin American Studies,Latin American Studies,African Studies,Latin American Studies,other option for 2nd period is TAing Inquiry a...
2,2017-06-14 15:39:51.924000,ktsapis@icsd.k12.ny.us,Abel Logan,7,Kaile,7th-8th Grade Art,Geography and Borders,7th-8th Grade Art,Spanish B,Spanish B,...,Researchers (8th Grade SS),Researchers (8th Grade SS),MS/HS PE,MS/HS PE,Poet's Society,Spanish B,Computer Literacy,Fiber Tech,Fiber Tech,Empty
3,Empty,jpaskin@icsd.k12.ny.us,Abeygail Sullivan-Pomeroy,5,?,6thGradeOther,6thGradeOther,6thGradeOther,6thGradeOther,6thGradeOther,...,6thGradeOther,6thGradeOther,6thGradeOther,Spanish A,Poet's Society,6th Grade Art,Fiber Tech,MS Chemistry,MS PE,Empty
4,2017-06-15 21:56:03.811000,nora.schapira@icsd.k12.ny.us,Abiah,12,Nora,Portfolio Photo,HS English (TBA),Other,Asia Studies,Drawing and Painting,...,Economics,Economics,Other,Other,Other,Other,Ecology,Ecology,Other,Period 7: Other: TA Spanish B (OK with me)\nWe...
5,2017-06-14 10:20:54.506000,sbokaer@icsd.k12.ny.us,Adina Wilensky,7,Sarah Jane Bokaer,French B,French B,French B,Algebra A,Global History of Commodities,...,MS/HS PE,Algebra A,MS Science (TBA),Poet's Society,MS/HS PE,MS/HS PE,MS Chemistry,MS Chemistry,Computer Literacy,"For Period 3, Dan Torrington has arranged for ..."
6,Empty,jpaskin@icsd.k12.ny.us,Adrian Dvorak,5,?,6thGradeOther,6thGradeOther,6thGradeOther,6thGradeOther,6thGradeOther,...,6thGradeOther,6thGradeOther,6thGradeOther,MS/HS PE,6th Grade Art,Spanish A,MS Chemistry,Fiber Tech,MS PE,Empty
7,2017-06-16 08:59:54.191000,michael.nardi@icsd.k12.ny.us,alana craib,10,michael,Facing History,Evolutionary Biology,Evolutionary Biology,Facing History,Asia Studies,...,Intermediate Algebra and Geometry,Intermediate Algebra and Geometry,Intermediate Algebra and Geometry,Molecular Biology,Celtic Band,US History and Lit: The 20th Century,Choir,Dark Fiction,US History and Lit: The 20th Century,Empty
8,2017-06-16 11:17:26.857000,bronwen.exter@icsd.k12.ny.us,Alannah McGarry,12,Bronwen,Constitutional Law/Government,Pre-Calculus,Constitutional Law/Government,HS English (TBA),HS English (TBA),...,Community Service Class,Economics,Community Service Class,Molecular Biology,US History and Lit: The 20th Century,Anatomy and Physiology,Dark Fiction,US History and Lit: The 20th Century,Dark Fiction,"To graduate this year, Alannah's schedule opti..."
9,2017-06-15 22:39:31.978000,nora.schapira@icsd.k12.ny.us,Alec Simmons,12,Nora,Pre-Calculus,Other,Pre-Calculus,Other,HS English (TBA),...,Economics,Economics,Other,Anatomy and Physiology,Molecular Biology,Anatomy and Physiology,Dark Fiction,Probability and Statistics,Dark Fiction,Other: Free Period


In [10]:
s = 0
for i in num_other:
    s += num_other[i]
avg = float(s)/len(num_other)
print("On average, each period/choice has", avg, "number of 'other'")

On average, each period/choice has 29.263157894736842 number of 'other'


#### (TODO) Check that all courses requested are in the correct period

#### Make preferences table

In [23]:
# make sure each entry is unique
len(clean_data['Student Name']) == len(set(clean_data['Student Name']))
# student_ids = np.core.defchararray.add('S', np.char.mod('%d', np.arange(len(clean_data['Student Name']))))

True

## Model 1 Data: 
#### Binary Matrix by Schedule
#### Course Caps

In [37]:
def all_classes(clean_data):
    # returns a list of all the class names in a period
    result = []
    for period in [1, 2, 3, 4, 7, 8]:
        classes = clean_data.filter(like='Period ' + str(period))
        classes = pd.get_dummies(classes)
        result.extend(classes.columns)
        for choice in ['First', 'Second', 'Third']:
            s = 'Period ' + str(period) + ' (' + choice + ' Choice)'
            s2 = 'P' + str(period)
            result = [c.replace(s, s2) for c in result]
    return result

def schedule_preferences(clean_data):
    # creates 3 csvs with each student's schedule preferences in binary format
    full_classes = all_classes(clean_data)
    for choice in ['First', 'Second', 'Third']:
        pref = clean_data.filter(like = choice + ' Choice')
        binary = pd.get_dummies(pref)
        cols = binary.columns
        # replace column names
        num_per = 7
        for i in [1, 2, 3, 4, 7, 8]:
            # add classes for each period that were not chosen
            s = 'Period ' + str(i) + ' (' + choice + ' Choice)'
            s2 = 'P' + str(i)
            cols = [c.replace(s, s2) for c in cols]
        empty = list(set(full_classes) - set(cols))
        binary.columns = cols
        for e in empty:
            binary[e] = np.zeros(len(clean_data))
        binary = binary.reindex(sorted(binary.columns), axis=1)
        binary.to_csv('resources/' + choice + 'ChoiceBinary.csv')

def course_caps(clean_data, class_size):
    n = 8
    period_courses = []
    for i in range(n):
        classes_in_period = clean_data.filter(like='Period ' + str(i + 1))
        classes_in_period = list(set(classes_in_period.values.flatten()))
        
        if 0 in classes_in_period:  # remove the NaNs we cast earlier
            classes_in_period.remove(0)
        
        courses = np.core.defchararray.add(np.repeat("P" + str(i + 1)+'_', len(classes_in_period)) , np.array(classes_in_period))
        period_courses.extend(courses)

    period_courses = pd.DataFrame(period_courses)

    period_courses['Size'] = np.repeat(class_size, len(period_courses))
    period_courses.to_csv('resources/CourseSize.csv')
        
# schedule_preferences(clean_data)
# course_caps(clean_data, 22)

## Model 2 Data: 
#### Binary Matrix (flat) with double/multiple periods considered
#### Course Caps with double/multiple periods considered

In [52]:
def flat_preferences(clean_data):
    # creates a single csv with student's course preferences in binary format
    full_classes = all_classes(clean_data)

    for i in [1, 2, 3, 4, 7, 8]:
        s = 'P' + str(i) + '_'
        full_classes = [c.replace(s, '') for c in full_classes]

    cols = set(full_classes)
    cols = list(cols)
    
    # add multiple course instances to course list
    mult = [("Algebra A", 2), ("Algebra B", 2), ("Beginning Algebra and Geometry", 2), ("Advanced Algebra and Trigonometry", 2), ("Intermediate Algebra and Geometry", 2), ("HS English (TBA)", 2), ("Spanish A", 2), ("Spanish B", 2), ("HS PE", 3), ("MS/HS PE", 3), ("Other", 6) ]
    for course in mult:
        for reps in range(course[1]-1): # for number of repetitions
            cols.append(course[0] + " " + str(reps + 2))  
    
    # add double periods to course list
    double = ["African Studies", "MS English and Social Studies", "Facing History", "American Studies/Global Perspective", "US History and Lit: The 20th Century", "Latin American Studies"]
    for course in double:
        cols.append(course + " " + str(2))
    
    
    flat = pd.DataFrame(index=np.arange(len(clean_data)), columns=cols)
    flat = flat.fillna(0)

    students = clean_data.filter(like='Choice')
    mult_pref = [c[0] for c in mult]
    mult_pref = mult_pref + double # list of all courses to extend student preferences to
    
    for idx, row in students.iterrows():
        for course in set(row[1:]):
            flat.at[idx, course] = 1
            # updated prefs to account for doubles and multiple instances
            if course in mult_pref: 
                # determine number of instances
                if course == "Other":
                    count = 6
                elif course == "HS PE" or course == "MS/HS PE":
                    count = 3
                else:
                    count = 2
                for rep in range(2, count + 1):
                    flat.at[idx, course + " " + str(rep)] = 1
        
    
    flat = flat.reindex(sorted(flat.columns), axis=1)        
    flat.to_csv('resources/FlatChoicesBinary.csv')

def flat_course_caps(clean_data, class_max, class_min):
    full_classes = all_classes(clean_data)
    for i in [1, 2, 3, 4, 7, 8]:
        s = 'P' + str(i) + '_'
        full_classes = [c.replace(s, '') for c in full_classes]
    
    cols = set(full_classes)
    cols = list(cols)
    
    # add multiple course instances to course list
    mult = [("Algebra A", 2), ("Algebra B", 2), ("Beginning Algebra and Geometry", 2), ("Advanced Algebra and Trigonometry", 2), ("Intermediate Algebra and Geometry", 2), ("MS Social Studies", 2), ("HS English (TBA)", 2), ("Spanish A", 2), ("Spanish B", 2), ("HS PE", 3), ("MS/HS PE", 3), ("Other", 6) ]
    for course in mult:
        for reps in range(course[1]-1): # for number of repetitions
            cols.append(course[0] + " " + str(reps + 2))  
    
    # add double periods to course list
    double = ["African Studies", "MS English and Social Studies (TBA)", "Facing History", "American Studies/Global Perspective", "US History and Lit: The 20th Century", "Latin American Studies"]
    for course in double:
        cols.append(course + " " + str(2))
    
    cols = sorted(cols)
    
    courses = pd.DataFrame(cols)
    courses['Max'] = np.repeat(class_max, len(courses))
    courses['Min'] = np.repeat(class_min, len(courses))
    courses.to_csv('resources/FlatCourseSize.csv')

flat_preferences(clean_data)
#flat_course_caps(clean_data, 22, 10)

### Old course-wise proximity function

In [193]:
def course_proximity():
    # note that "IV"/"G" and "IV"/"D" are equiv by "G"/"D" are not
    equiv =[ ("IIA", "G"), ("IIB", "F"), ("IIC", "K"), ("VI", "I"), ("IV", "D"), ("IV", "G"), ("VII", "E")]

    depts = pd.read_csv('resources/FlatCourseSize.csv', delimiter=',')
    sim = pd.DataFrame(index=depts['0'], columns=depts['0'])
    sim = sim.fillna(0)

    # depts = depts[["0", "MS Category", "HS Category"]]
    depts = depts.fillna(0)

    # iterate over depts
    msd = set(depts["MS Category"])
    msd.remove(0)

    hsd = set(depts["HS Category"])
    hsd.remove(0)

    ms_equiv = np.array([pair[1] for pair in equiv])
    hs_equiv = np.array([pair[0] for pair in equiv])

    for d in msd:
        dept_courses = depts["0"].loc[depts["MS Category"] == d]

        # find equiv dept
        if d in ms_equiv:
            equiv_depts = hs_equiv[np.where(ms_equiv == d)[0]]
            for ed in equiv_depts:    
                dept_courses = dept_courses.append(depts["0"].loc[depts["HS Category"] == ed], ignore_index=True)

        for course in dept_courses:
            sim.loc[course, dept_courses]=1

    for d in hsd:
        dept_courses = depts["0"].loc[depts["HS Category"] == d]

        # find equiv dept
        if d in hs_equiv:
            equiv_depts = ms_equiv[np.where(hs_equiv == d)[0]]
            for ed in equiv_depts:    
                dept_courses = dept_courses.append(depts["0"].loc[depts["MS Category"] == ed], ignore_index=True)

        for course in dept_courses :
            sim.loc[course, dept_courses]=1

    sim.to_csv('resources/Proximity.csv')

### Updated department-wise proximity matrix

In [51]:
# check that data is cleaned
classes = clean_data.filter(like='Period')

temp = []
for c in classes:
    temp.extend(classes[c])
set(temp)

{'6th Grade Art',
 '6thGradeOther',
 '7th-8th Grade Art',
 '8th Grade Science',
 'Adaptive PE',
 'Advanced Algebra and Trigonometry',
 'Advanced Spanish',
 'Advanced/In-Depth French',
 'African Studies',
 'Algebra A',
 'Algebra B',
 'American Studies/Global Perspective',
 'Anatomy and Physiology',
 'Asia Studies',
 'Banned Books',
 'Beginning Algebra and Geometry',
 'Beginning Algebra and Trigonometry',
 'Celtic Band',
 'Chemistry',
 'Choir',
 'Community Service Class',
 'Computer Literacy',
 'Constitutional Law/Government',
 'Dark Fiction',
 'Drawing and Painting',
 'Ecology',
 'Economics',
 'Empty',
 'English Seminar',
 'Evolutionary Biology',
 'Facing History',
 'Fiber Tech',
 'French A',
 'French B',
 'Geography and Borders',
 'Geology',
 'Global History of Commodities',
 'HS English (TBA)',
 'HS Health',
 'HS PE',
 'Health',
 'I Know Some Things',
 'In-Depth Spanish',
 'Intermediate Algebra and Geometry',
 'Intermediate French',
 'Intermediate Spanish',
 'Intro to Physics and Chem

In [90]:
def dept_proximity():
    # columns are department, rows are course names

    depts = pd.read_csv('resources/FlatCourseSize.csv', delimiter=',')

    # depts = depts[["0", "MS Category", "HS Category"]]
    depts = depts.fillna(0)

    # iterate over depts
    msd = set(depts["MS Category"])
    msd.remove(0)

    hsd = set(depts["HS Category"])
    hsd.remove(0)

    cols = msd.union(hsd)
    single = [subject for subject in cols if '&' not in subject]
    double = [subject for subject in cols if '&' in subject]

    sim = pd.DataFrame(index=depts['0'], columns=single)

    for d in cols:  
        dept_courses_MS = depts["0"].loc[depts["MS Category"] == d]
        dept_courses_HS = depts["0"].loc[depts["HS Category"] == d]
        dept_courses = dept_courses_MS.append(dept_courses_HS, ignore_index=True)
        for course in dept_courses:
            if d in single: 
                sim.loc[course, d]=1

            else: # d in double
                # get two depts
                d1 = d.split(" ")[0]
                d2 = d.split(" ")[2]

                sim.loc[course, d1]=1
                sim.loc[course, d2]=1

    sim = sim.fillna(0)            
    sim.to_csv('resources/Proximity.csv')

dept_proximity()

In [22]:
first = pd.read_csv('resources/FirstChoiceBinary.csv', delimiter=',')
second = pd.read_csv('resources/SecondChoiceBinary.csv', delimiter=',')
third = pd.read_csv('resources/ThirdChoiceBinary.csv', delimiter=',')

print(len(first.columns))
print(len(second.columns))
print( len(third.columns))

110
110
110


In [192]:
clean_data['Grade'].to_csv('resources/Grades.csv')